In [5]:
!pip install  sentence_transformers elasticsearch==9.0.0 emoji

In [6]:
!pip install numpy

In [7]:
import pandas as pd
data=pd.read_pickle("cleaned_clustered_data.pkl")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_clustered_data.pkl'

In [ ]:
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

def clean_text(text):
    if not isinstance(text, str):
        return ""
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters, punctuation, and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')  # Removes all emojis
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back into a single string
    text = ' '.join(tokens)
    return text

# Apply the updated function
data['detailed_description'] = data['detailed_description'].apply(clean_text)

In [ ]:
data['detailed_description'].head()

In [ ]:
data.info()

In [ ]:
# Convert platform columns to boolean
platform_columns = ['is_free','platforms_windows', 'platforms_mac', 'platforms_linux']

# Convert each platform column to boolean
for col in platform_columns:
    # Convert to boolean, handling any non-boolean values
    data[col] = data[col].map({'True': True, 'False': False, True: True, False: False})

    # Fill any remaining NaN values with False
    data[col] = data[col].fillna(False)

    # Convert to boolean type
    data[col] = data[col].astype(bool)

# Verify the changes
print("\nPlatform columns data types:")
print(data[platform_columns].dtypes)

print("\nPlatform columns value counts:")
for col in platform_columns:
    print(f"\n{col}:")
    print(data[col].value_counts())

In [ ]:
data.drop(columns=['_id'],inplace=True)

In [ ]:
data.info()

In [ ]:
def convert_price(price):
    if price == 'Free':
        return 0  # Convert 'Free' to 0
    elif isinstance(price, str) and '$' in price:
        return price.replace('$', '').strip()  # Remove '$' and whitespace
    else:
        return price  # Keep other values as is

data['price_final'] = data['price_final'].apply(convert_price)

# Convert to numeric, handling errors
data['price_final'] = pd.to_numeric(data['price_final'], errors='coerce')

# Fill NaN values with 0 (if necessary)
data['price_final'] = data['price_final'].fillna(0)

data['price_final'].head()

In [ ]:
data.isnull().sum()

In [ ]:
data.fillna('',inplace=True)

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], errors='coerce')

In [ ]:
data['required_age'] = pd.to_numeric(data['required_age'], errors='coerce')

In [ ]:
import ast

def convert_to_list(languages_str):
    try:
        # Attempt to evaluate the string as a Python literal
        languages_list = ast.literal_eval(languages_str)

        # Check if the result is a list
        if isinstance(languages_list, list):
            return languages_list
        else:
            return []  # Return an empty list if not a list
    except (SyntaxError, ValueError):
        # Handle errors if the string is not a valid Python literal
        return []

# Apply the function to the 'supported_languages' column
data['supported_languages'] = data['supported_languages'].apply(convert_to_list)

In [ ]:
data.info()

In [ ]:
data['supported_languages'].head()

In [ ]:
data['release_date'].head()

In [9]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from datetime import datetime
import pandas as pd
from tqdm import tqdm

# Initialize Elasticsearch
es = Elasticsearch("https://4ca02f04fab74e44a2ab8c97bfec0bd0.asia-southeast1.gcp.elastic-cloud.com:443",api_key="T2VwanhaWUJDZGNLQUpCV1FSQXA6cGtVNDVybFduVFdmRzA5dFNKTElRUQ==")

# Load embedding model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Create the index if not exists
index_name = "games"

mapping = {
  "mappings": {
    "properties": {
      "app_id":               { "type": "integer" },
      "name":                 { "type": "text", "fields": { "raw": { "type": "keyword" } } },
      "type":                 { "type": "keyword" },
      "required_age":         { "type": "float" },
      "is_free":              { "type": "boolean" },
      "supported_languages":  { "type": "keyword" },
      "developers":           { "type": "keyword" },
      "publishers":           { "type": "keyword" },
      "price_final":          { "type": "float" },
      "platforms_windows":    { "type": "boolean" },
      "platforms_mac":        { "type": "boolean" },
      "platforms_linux":      { "type": "boolean" },
      "categories":           { "type": "keyword" },
      "genres":               { "type": "keyword" },
      "recommendations_total":{ "type": "keyword" },
      "release_date":         { "type": "date","format": "yyyy-MM-dd"},
      "price_currency":       { "type": "keyword" },
      "metacritic_score":     { "type": "keyword" },
      "short_description":    { "type": "text" },
      "detailed_description": { "type": "text" },
      "about_the_game":       { "type": "text" },
      "website":              { "type": "keyword" },
      "discount_percent":     { "type": "keyword" },
      "pc_min_os":            { "type": "text" },
      "pc_min_processor":     { "type": "text" },
      "pc_min_memory":        { "type": "text" },
      "pc_min_graphics":      { "type": "text" },
      "pc_min_directx":       { "type": "text" },
      "pc_min_network":       { "type": "text" },
      "pc_min_storage":       { "type": "text" },
      "header_image":         { "type": "keyword" },
      "background":           { "type": "keyword" },
      "screenshots_count":    { "type": "keyword" },
      "movies_count":         { "type": "keyword" },
      "pc_rec_os":            { "type": "text" },
      "pc_rec_processor":     { "type": "text" },
      "pc_rec_memory":        { "type": "text" },
      "pc_rec_graphics":      { "type": "text" },
      "support_email":        { "type": "keyword" },
      "pc_rec_directx":       { "type": "text" },
      "pc_rec_network":       { "type": "text" },
      "pc_rec_storage":       { "type": "text" },
      "cluster":              { "type": "integer" },
      "description_vector":   { "type": "dense_vector", "dims": 384 }
    }
  }
}


if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=mapping)
    print(f"Created index: {index_name}")
else:
    print(f"Index {index_name} already exists")

# Prepare document
# Prepare document
def prepare_document(row):

    # Compute embedding vector for detailed_description
    description = row['detailed_description'] if isinstance(row['detailed_description'], str) else ""
    description_vector = model.encode(description).tolist()

    doc = {
        "app_id": int(row['app_id']),
        "name": row['name'],
        "type": row['type'],
        # Convert 'required_age' to a valid numeric value if it's NaN
        "required_age": float(row['required_age']) if pd.notna(row['required_age']) else 0,
        "is_free": bool(row['is_free']),
        "supported_languages": row['supported_languages'],
        "developers": row['developers'],
        "publishers": row['publishers'],
        "price_final": float(row['price_final']),
        "platforms_windows": bool(row['platforms_windows']),
        "platforms_mac": bool(row['platforms_mac']),
        "platforms_linux": bool(row['platforms_linux']),
        "categories": row['categories'],
        "genres": row['genres'],
        "recommendations_total": row['recommendations_total'],
         "release_date": row['release_date'].strftime('%Y-%m-%d') if pd.notna(row['release_date']) else None,
        "price_currency": row['price_currency'],
        "metacritic_score": row['metacritic_score'],
        "short_description": row['short_description'],
        "detailed_description": description,
        "about_the_game": row['about_the_game'],
        "website": row['website'],
        "discount_percent": row['discount_percent'],
        "pc_min_os": row['pc_min_os'],
        "pc_min_processor": row['pc_min_processor'],
        "pc_min_memory": row['pc_min_memory'],
        "pc_min_graphics": row['pc_min_graphics'],
        "pc_min_directx": row['pc_min_directx'],
        "pc_min_network": row['pc_min_network'],
        "pc_min_storage": row['pc_min_storage'],
        "header_image": row['header_image'],
        "background": row['background'],
        "screenshots_count": row['screenshots_count'],
        "movies_count": row['movies_count'],
        "pc_rec_os": row['pc_rec_os'],
        "pc_rec_processor": row['pc_rec_processor'],
        "pc_rec_memory": row['pc_rec_memory'],
        "pc_rec_graphics": row['pc_rec_graphics'],
        "support_email": row['support_email'],
        "pc_rec_directx": row['pc_rec_directx'],
        "pc_rec_network": row['pc_rec_network'],
        "pc_rec_storage": row['pc_rec_storage'],
        "cluster": int(row['cluster']),
        "description_vector": description_vector
    }

    return doc

# Index data
def index_data(data):
    for idx, row in tqdm(data.iterrows(), total=len(data)):
        doc = prepare_document(row)
        es.index(index=index_name, id=int(row.name), body=doc)
    print("Indexing completed!")

# Run
if __name__ == "__main__":
    index_data(data)

In [18]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import json
index_name = "games"

# --- Vector Encoder ---
def encode_query(text):
    return model.encode(text).tolist()

# --- Search Functions ---
def search_bm25(query_text, size=100, filtered_ids=None):
    bm25_query = {
        "size": size,
        "query": {
            "bool": {
                "must": [
                    {"match": {"detailed_description": query_text}}
                ]
            }
        }
    }

    if filtered_ids:
        bm25_query["query"]["bool"]["filter"] = {
            "ids": {"values": list(filtered_ids)}
        }

    res = es.search(index=index_name, body=bm25_query)
    return {hit["_id"]: hit["_score"] for hit in res["hits"]["hits"]}

def search_vector(query_vector, k=100, candidates=200, filtered_ids=None):
    if filtered_ids:
        # Fallback: manually filter after search (ES does not support knn+ids directly)
        vector_query = {
            "knn": {
                "field": "description_vector",
                "query_vector": query_vector,
                "k": candidates,
                "num_candidates": candidates
            }
        }
        res = es.search(index=index_name, body=vector_query)
        return {
            hit["_id"]: hit["_score"]
            for hit in res["hits"]["hits"] if hit["_id"] in filtered_ids
        }

    else:
        vector_query = {
            "knn": {
                "field": "description_vector",
                "query_vector": query_vector,
                "k": k,
                "num_candidates": candidates
            }
        }
        res = es.search(index=index_name, body=vector_query)
        return {hit["_id"]: hit["_score"] for hit in res["hits"]["hits"]}

# --- Normalize Scores ---
def normalize(scores_dict):
    ids = list(scores_dict.keys())
    scores = np.array(list(scores_dict.values())).reshape(-1, 1)
    normalized = MinMaxScaler().fit_transform(scores).flatten()
    return dict(zip(ids, normalized))

# --- Combine Scores ---
def combine_scores(bm25_scores, vec_scores, weight_bm25=0.5, weight_vec=0.5):
    norm_bm25 = normalize(bm25_scores)
    norm_vec = normalize(vec_scores)

    combined = {}
    for _id in set(norm_bm25.keys()).union(norm_vec.keys()):
        bm25 = norm_bm25.get(_id, 0)
        vec = norm_vec.get(_id, 0)
        combined[_id] = weight_bm25 * bm25 + weight_vec * vec
    return combined

# --- Final Hybrid Search ---
import json

def hybrid_search(query_text, top_k=10, filtered_ids=None):
    print(f"\nSearching for: {query_text}")

    query_vector = encode_query(query_text)
    bm25_scores = search_bm25(query_text, filtered_ids=filtered_ids)
    vec_scores = search_vector(query_vector, filtered_ids=filtered_ids)

    combined = combine_scores(bm25_scores, vec_scores)
    top_results = sorted(combined.items(), key=lambda x: x[1], reverse=True)[:top_k]

    results = []
    for rank, (_id, score) in enumerate(top_results, start=1):
        doc = es.get(index=index_name, id=_id)
        source = doc["_source"]

        filtered_source = {
            k: v for k, v in source.items()
            if k not in ["cluster", "description_vector"]
        }

        result = {
            "rank": rank,
            "score": round(score, 2),
            **filtered_source
        }

        results.append(result)

    return json.dumps(results, indent=2, ensure_ascii=False)

# --- Direct Search by Name ---
def direct_elastic(name):
    query = {
        "query": {
            "term": {
                "name.raw": name
            }
        }
    }

    res = es.search(index=index_name, body=query)

    results = []
    for hit in res["hits"]["hits"]:
        source = hit["_source"]
        filtered_source = {
            k: v for k, v in source.items()
            if k not in ["cluster", "description_vector"]
        }
        results.append(filtered_source)

    return json.dumps(results, ensure_ascii=False)

# --- Optional Filter Function ---
def filter_elastic(filters,use_cluster=True):
    query = {
        "query": {
            "bool": {
                "must": []
            }
        }
    }

    # Year Range Filter
    if filters.get("year_range"):
        year_range = filters["year_range"]
        if isinstance(year_range, str) and "-" in year_range:
            try:
                start_year, end_year = map(int, year_range.split("-"))
            except ValueError:
                start_year = end_year = int(year_range)
        else:
            start_year = end_year = int(year_range)

        query["query"]["bool"]["must"].append({
            "range": {
                "release_date": {
                    "gte": f"{start_year}-01-01",
                    "lte": f"{end_year}-12-31"
                }
            }
        })

    # Developer Filter
    if filters.get("developer"):
        query["query"]["bool"]["must"].append({
            "term": {
                "developers": filters["developer"]
            }
        })

    # Publisher Filter
    if filters.get("publisher"):
        query["query"]["bool"]["must"].append({
            "term": {
                "publishers": filters["publisher"]
            }
        })

    # Game Description (full-text search)
    if filters.get("game_description"):
        query["query"]["bool"]["must"].append({
            "match": {
                "detailed_description": filters["game_description"]
            }
        })

    # Cluster Filter
    if use_cluster and filters.get("cluster"):
        query["query"]["bool"]["must"].append({
            "term": {
                "cluster": filters["cluster"]
            }
        })

    # Platform Filter (windows/mac/linux)
    if filters.get("platform"):
        platform_field = f"platforms_{filters['platform'].lower()}"
        query["query"]["bool"]["must"].append({
            "term": {
                platform_field: True
            }
        })

    # Currency Filter
    if filters.get("currency"):
        query["query"]["bool"]["must"].append({
            "term": {
                "price_currency": filters["currency"]
            }
        })

    # Price Limit
    if filters.get("price_limit"):
        try:
            price = float(filters["price_limit"])
            query["query"]["bool"]["must"].append({
                "range": {
                    "price_final": {
                        "lte": price
                    }
                }
            })
        except ValueError:
            pass

    # Language Filter
    if filters.get("language"):
      query["query"]["bool"]["must"].append({
        "term": {
            "supported_languages.keyword": [filters["language"]]
        }
    })

    query["size"] = 1000
    res = es.search(index=index_name, body=query)

    return [hit["_id"] for hit in res["hits"]["hits"]]


def get_games(results):
    func_name=results.get("function_name")
    if func_name=="direct_search":
        name=results.get("name")
        return direct_elastic(name)
    elif func_name=="filter_search":
        result_filter = filter_elastic(results, use_cluster=True)
        print(result_filter)
        if not result_filter or len(result_filter) < 5:
          result_filter = filter_elastic(results, use_cluster=False)
        query_text=results.get("game_description")
        return hybrid_search(query_text,filtered_ids=result_filter)
    else:
        return results.get("response")

In [13]:
!pip install dotenv google

In [14]:
from LLM_Response import LLMResponse
from google import genai
import os
from dotenv import load_dotenv
load_dotenv('api.env')
api_key = os.environ.get('gemini_api_key')
genai_client =genai.Client(api_key=api_key)
llm=LLMResponse(api_key)

Function to call: direct_search
Arguments: {'game_name': 'Counter-Strike 2'}
Direct Search Result: {'function_name': 'direct_search', 'game_name': 'Counter-Strike 2'}
Function to call: filter_search
Arguments: {'year_range': '2022-2025', 'cluster': '12', 'price_limit': '0', 'game_description': 'single-player racing game', 'language': 'English'}
Filter Search Result: {'function_name': 'filter_search', 'game_description': 'single-player racing game', 'developer': None, 'publisher': None, 'year_range': '2022-2025', 'price_limit': '0', 'cluster': '12', 'platform': None, 'currency': None}
Function to call: chit_chat
Arguments: {'query': 'Hello, how are you?'}
Chit Chat Result: {'function_name': 'chit_chat', 'response': 'I am doing well, thank you for asking! How are you today?\n'}
Reflected Query: I want a free racing game that supports multiplayer



In [15]:
query_text="I want to play a single-player racing game with high end graphics that from support Spanish"
result=llm.get_function_call(query_text)
print(result)

Function to call: filter_search
Arguments: {'language': 'Spanish', 'game_description': 'single-player racing game with high end graphics', 'cluster': '12'}
{'function_name': 'filter_search', 'game_description': 'single-player racing game with high end graphics', 'developer': None, 'publisher': None, 'year_range': None, 'price_limit': None, 'cluster': '12', 'platform': None, 'currency': None}


In [19]:
game=get_games(result)
print(game)

['1833', '4611', '4950', '3069', '3547', '2525', '3519', '4112', '4019', '1393', '2174', '2875', '1201', '2535', '5120', '4463', '182', '2286', '2667', '1605', '3733', '2725', '4589', '2533', '2256', '1906', '5159', '321', '1270', '2462', '3099', '2162', '1278', '2310', '1954', '5162', '4430', '4797', '3147', '2343', '296', '5144', '4000', '570', '2953', '4624', '1356', '2092', '1086', '5032', '3160', '4154', '4100', '3887', '290', '1714', '2350', '4628', '3163', '3959', '4205', '4770', '4210', '516', '986', '1642', '3503', '4004', '2262', '4056', '4089', '4514', '3316', '495', '4709', '3699', '4443', '4964', '3736', '4033', '3537', '1239', '3194', '2999', '858', '1924', '4425', '1426', '4905', '4892', '2785', '2935', '1697', '4850', '4140', '3149', '3460', '1439', '3295', '5102', '2280', '3738', '2011', '3032', '451', '4136', '3729', '1290', '4111', '4418', '3233', '4046', '636', '5017', '5052', '2660', '2370', '662', '4629', '4690', '4932', '2662', '2238', '2229', '4607', '1735', '13